In [3]:
# Import relevant functionality
# from langchain_anthropic import ChatAnthropic
from langchain_ollama import ChatOllama
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent


In [4]:
import os
import getpass

os.environ["TAVILY_API_KEY"] = "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"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [ ]:
# Create the agent
memory = MemorySaver()
# model = ChatAnthropic(model_name="claude-3-sonnet-20240229")
llm = ChatOllama(
    model = "llama3.2:1b",
    temperature = 0.8,
    num_predict = 256,
    disable_streaming=False

)
messages = [ ("system", "You are a math professor. Your job is to solve the user's math problems"), ("user", "What is an eigevalue")]
response = llm.invoke(messages)
print(response)
search = TavilySearchResults(
    max_results=5,
    include_answer=True,
    include_raw_content=True,
    include_images=True)

tools = [search]
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}} # specify therad id to it agent knows from which thread/conversation to load state from
# for chunk in agent_executor.stream(
#     {"messages": [HumanMessage(content="hi im bob! and i live in ottawa")]}, config
# ):
#     print(chunk)
#     print("----")



In [ ]:
messages = [
    ("human", "Return the words Hello World!"),
]
for chunk in llm.stream(messages):
    print(chunk)

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")

In [ ]:
async for event in agent_executor.astream_events(
    {"messages": [SystemMessage(content="You are a helpful assistant"), HumanMessage(content="where should I move from where I live currently if I want weather that is the same as where I am now?")]}, version="v1", config=config
):
    print(event)
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        print("on model stream")
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

In [6]:
os.environ["GROQ_API_KEY"] = getpass.getpass()

from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-8b-8192")

In [9]:
llm = ChatGroq(model="llama3-8b-8192")
search = TavilySearchResults(
    max_results=5,
    include_answer=True,
    include_raw_content=True,
    include_images=True)

tools = [search]
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

In [13]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="who is blackpink rose number one girl?")]}, version="v2", config=config
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")



--
Starting tool: tavily_search_results_json with inputs: {'query': 'blackpink rose number one girl'}
Done tool: tavily_search_results_json
Tool output was: content='[{"url": "https://kbizoom.com/rose-number-one-girl-lyrics-meaning-and-song-credits/", "content": "A Korean media outlet has confirmed that BLACKPINK\'s Rosé will perform \\"number one girl\\" for the first time at the 2024 MAMA AWARDS, which will be held at the Kyocera Dome in Osaka, Japan, on November 22 (KST).. Composed solely of ROSÉ\'s voice and piano accompaniment, \\"number one girl\\" showcases her unparalleled vocal abilities. The rich emotional expression and poignant appeal"}, {"url": "https://apnews.com/article/blackpink-rose-rosie-kpop-music-review-fe095125d478df249be8c58343bbedb3", "content": "Rosé, the nightingale of the massively popular Korean girl group Blackpink, has flown the nest and launched her first solo album \\"Rosie.\\" ... The album starts off chipper, with the soaring piano ballad \\"Number One 